# 1. 写一个正则表达式，使其能同时识别下面所有的字符串：'bat','bit', 'but', 'hat', 'hit', 'hut

首先在此处导入所有需要的库:

In [8]:
import re
import requests
from lxml import etree
import difflib

现在编写正则表达式

In [9]:
# 设置待匹配字符串
s_1 = 'bat,bit, but, hat, hit, hut'

# 设置pattern
patt_1 = re.compile( r'[b|h].t')

#返回结果
result_1 = patt_1.findall(s_1)

print(result_1)

['bat', 'bit', 'but', 'hat', 'hit', 'hut']


# 2. 提取下列字符串中完整的年月日和时间字段；s="我出生时间为1990-01-01 00:00:00,今天时间为2019-04-20 12:20:00"

In [10]:
s = "我出生时间为1990-01-01 00:00:00,今天时间为2019-04-20 12:20:00"

patt_2 = re.compile(r'\d+-\d+-\d+\s\d+:\d+:\d+')

result_2 = patt_2.findall(s)

print(result_2)

['1990-01-01 00:00:00', '2019-04-20 12:20:00']


# 3. 找出下列字符串的歌手和歌名
``` html
html = '''
<div id="songs-list">
    <h2 class="title">经典老歌</h2>
    <p class="introduction">
        经典老歌列表
    </p>
    <ul id="list" class="list-group">
        <li data-view="2">一路上有你</li>
        <li data-view="7">
            <a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
        </li>
        <li data-view="4" class="active">
            <a href="/3.mp3" singer="齐秦">往事随风</a>
        </li>
        <li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
        <li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
        <li data-view="5">
            <a href="/6.mp3" singer="邓丽君">但愿人长久</a>
        </li>
    </ul>
</div>
'''
```

In [11]:

html = '''
<div id="songs-list">
    <h2 class="title">经典老歌</h2>
    <p class="introduction">
        经典老歌列表
    </p>
    <ul id="list" class="list-group">
        <li data-view="2">一路上有你</li>
        <li data-view="7">
            <a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
        </li>
        <li data-view="4" class="active">
            <a href="/3.mp3" singer="齐秦">往事随风</a>
        </li>
        <li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
        <li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
        <li data-view="5">
            <a href="/6.mp3" singer="邓丽君">但愿人长久</a>
        </li>
    </ul>
</div>
'''


patt_3a = re.compile('<a href=".*?" singer=".*?">.*?</a>')

result_3a = patt_3a.findall(html)

print(result_3a)

['<a href="/2.mp3" singer="任贤齐">沧海一声笑</a>', '<a href="/3.mp3" singer="齐秦">往事随风</a>', '<a href="/4.mp3" singer="beyond">光辉岁月</a>', '<a href="/5.mp3" singer="陈慧琳">记事本</a>', '<a href="/6.mp3" singer="邓丽君">但愿人长久</a>']


In [12]:
result_3 = []

for i in result_3a:
    temp = {}
    temp['singer'] = re.findall(r'(?<=singer=").*?(?=")',i)[0]
    temp['name'] = re.findall(r'(?<=singer="{}">).*?(?=<)'.format(temp['singer']),i)[0]
    result_3 += [temp]
    
print(result_3)

[{'singer': '任贤齐', 'name': '沧海一声笑'}, {'singer': '齐秦', 'name': '往事随风'}, {'singer': 'beyond', 'name': '光辉岁月'}, {'singer': '陈慧琳', 'name': '记事本'}, {'singer': '邓丽君', 'name': '但愿人长久'}]


# 4. 打开京东官网，搜索python书，查看网页源代码，输出5本python书名，价格，出版社；

将所需代码封装成类如下：

In [13]:
class jingd():
    def __init__(self,keywored):
        self.keyword = keywored
        self.url = 'https://search.jd.com/Search'
        self.headers = {
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70'
            }
        self.params = {
                'keyword': keywored
            }


    def get_data(self,url):
        response = requests.get(url = url,headers = self.headers,params = self.params,timeout = 10)
        
        return response.content
    
    def parse_data(self,data):
        data_ed = re.sub(r'<font class="skcolor_ljg">'+self.keyword+r'</font>', self.keyword, data.decode(),flags = re.I)
        html = etree.HTML(data_ed)
        result = html.xpath(r'//*[@id="J_goodsList"]/ul/li')
        result_list =[] 
        for li in result:
            temp = {}
            name = li.xpath(r'./div/div[contains(@class,"p-name")]/a/em/text()')
            if len(name) == 0:
                name = li.xpath(r'.//div[contains(@class,"tab-content-item")]//div[contains(@class,"p-name")]/a/em/text()')
            temp['name'] = name[0] if len(name) == 1 else str(name)
            
            price = li.xpath(r'./div/div[contains(@class,"p-price")]/strong/i/text()')
            price_temp = li.xpath(r'.//div[contains(@class,"tab-content-item")]//div[contains(@class,"p-price")]//i/text()')
            temp['price'] = price[0] if  len(price) > len(price_temp) else str(price_temp)



            bi_store = li.xpath(r'./div/div[contains(@class,"p-bookdetails")]//span[contains(@class,"p-bi-store")]/a/text()')
            if len(bi_store) == 0:
                bi_store = li.xpath(r'.//div[contains(@class,"tab-content-item")]//span[contains(@class,"p-bi-store")]/a/text()')
            temp['bi_store'] = bi_store[0] if len(bi_store) == 1 else str(bi_store)


            result_list += [temp]
            
        
        return result_list
        
    def run(self):
        self.html = self.get_data(self.url)
        self.resultlist = self.parse_data(self.html)


In [14]:
# 设置搜索关键词并运行
result_4inp = jingd('python')
result_4inp.run()

In [15]:
# 打印网页内容
print(result_4inp.html.decode())

<!DOCTYPE html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="renderer" content="webkit">
<meta http-equiv="Cache-Control" content="max-age=300" />
<link rel="dns-prefetch" href="//search.jd.com" />
<link rel="dns-prefetch" href="//item.jd.com" />
<link rel="dns-prefetch" href="//list.jd.com" />
<link rel="dns-prefetch" href="//p.3.cn" />
<link rel="dns-prefetch" href="//misc.360buyimg.com" />
<link rel="dns-prefetch" href="//nfa.jd.com" />
<link rel="dns-prefetch" href="//d.jd.com" />
<link rel="dns-prefetch" href="//img12.360buyimg.com" />
<link rel="dns-prefetch" href="//img13.360buyimg.com" />
<link rel="dns-prefetch" href="//static.360buyimg.com" />
<link rel="dns-prefetch" href="//csc.jd.com" />
<link rel="dns-prefetch" href="//mercury.jd.com" />
<link rel="dns-prefetch" href="//x.jd.com" />
<link rel="dns-prefetch" href="//wl.jd.com" />
<title>python - 商品搜索 - 京东</title>
<meta name="K

In [16]:
# 打印获得的前五条内容
print(str(result_4inp.resultlist[:5]))

[{'name': "['python编程 从入门到实践 第2版（图灵出品）', 'python编程三剑客：python编程从入门到实践+python编程快速上手+python极客项目编程（京东套装共3册）', '精通python编程两件套：python编程从入门到实践+python数据结构与算法分析 第2版（京东套装共2册）']", 'price': "['104.00']", 'bi_store': '人民邮电出版社'}, {'name': 'python编程三剑客新版：python编程从入门到实践第2版+快速上手第2版+极客编程（套装3册）', 'price': '253.70', 'bi_store': '人民邮电出版社'}, {'name': '青少年编程入门、进阶、高阶——Scratch+python+C语言 趣味编程儿童编程教孩子学编程编程真好玩父与子的编程之旅', 'price': '250.20', 'bi_store': '[]'}, {'name': "['零基础学python（python3.10全新升级）（基础入门 同步视频 技术答疑）', 'python黄金学习组合（京东套装共4册）', 'python入门与实战组合套装（京东套装共2册）']", 'price': "['39.90']", 'bi_store': '吉林大学出版社'}, {'name': "['python学习手册（套装上下册）（原书第5版）', '零基础python程序员修炼之道:python学习手册+代码整洁之道（京东套装共3册）', 'python编程从入门到精通:python学习手册+代码整洁之道+标准库（京东套装共4册）']", 'price': "['219.00']", 'bi_store': '机械工业出版社'}]


# 5.	写一个函数查重，完成以下功能
（1）	输入：两份你们的作业py格式，

（2）	输出：两份作业的相似性
    
    做法1. 统计code中的单词数，生成一个单词数的列表计算相似性
    做法2. 采用正则表达式，对code进行关键词分割，按照关键词建立相似性


设py函数如下：

In [17]:
str_5_1 = '''
        def create_inverted_table(movie_profile):
            inverted_table = {}
            for mid, weights in movie_profile["weights"].iteritems():
                for tag, weight in weights.items():
                    _ = inverted_table.get(tag, [])
                    _.append((mid, weight))
                    inverted_table.setdefault(tag, _)
            return inverted_table
'''

str_5_2 = '''
from gensim.models import TfidfModel

import pandas as pd
import numpy as np

from pprint import pprint

# ......

def create_movie_profile(movie_dataset):
-
    dataset = movie_dataset["tags"].values

    from gensim.corpora import Dictionary
    # 根据数据集建立词袋，并统计词频，将所有词放入一个词典，使用索引进行获取
    dct = Dictionary(dataset)
    # 根据将每条数据，返回对应的词索引和词频
    corpus = [dct.doc2bow(line) for line in dataset]
    # 训练TF-IDF模型，即计算TF-IDF值
    model = TfidfModel(corpus)

    _movie_profile = []
    for i, data in enumerate(movie_dataset.itertuples()):
        mid = data[0]
        title = data[1]
        genres = data[2]
        vector = model[corpus[i]]
        movie_tags = sorted(vector, key=lambda x: x[1], reverse=True)[:30]
        topN_tags_weights = dict(map(lambda x: (dct[x[0]], x[1]), movie_tags))
        # 将类别词的添加进去，并设置权重值为1.0
        for g in genres:
            topN_tags_weights[g] = 1.0
        topN_tags = [i[0] for i in topN_tags_weights.items()]
        _movie_profile.append((mid, title, topN_tags, topN_tags_weights))

    movie_profile = pd.DataFrame(_movie_profile, columns=["movieId", "title", "profile", "weights"])
    movie_profile.set_index("movieId", inplace=True)
    return movie_profile

movie_dataset = get_movie_dataset()
pprint(create_movie_profile(movie_dataset))


'''

之后用正则表达式分词，最后用difflib库相关函数计算相似度

In [18]:
patt_5 = re.compile(r'[A-Za-z0-9]+')

result_5_1 = patt_5.findall(str_5_1)
result_5_2 = patt_5.findall(str_5_2)
def stri_similar(s1,s2):
    return difflib.SequenceMatcher(None,s1,s2).quick_ratio()

print(stri_similar(result_5_1,result_5_2))

0.1890547263681592
